# Fase 2: Limpieza y Transformación

#### 1. Importamos el fichero csv que contiene la data transformada, y añadimos low_memory para evitar que salga la advertencia con el tipo de datos mezclados. Cambiaremos el tipo de dato más adelante

In [1]:
import pandas as pd
df = pd.read_csv('../data/transform_data/transform_Data.csv', low_memory=False)

#### 2. Analizamos si existen o no valores duplicados, para poder reducir el tamaño del archivo y mejorar la velocidad del fichero

In [2]:
# Comprobamos si existen valores duplicados
if df.duplicated().values.any():
    print("Existen valores duplicados por eliminar")
else:
    print("No existen valores duplicados")

# Comprobamos y verificamos que no existen valores duplicados
print(df.duplicated().sum())

No existen valores duplicados
0


#### 3. Analizamos y eliminamos los valores nulos existentes, con el fin de obtener un set de datos lo más eficiente posible

In [3]:
# Calculamos cuantos valores nulos existen
print("Los valores nulos existentes por columna son:")
print(df.isnull().sum())

Los valores nulos existentes por columna son:
Unnamed: 0               0
age                  48290
job                  43515
marital              43255
education            44977
default              52151
housing              44196
loan                 44196
contact              43170
duration             43170
campaign             43170
pdays                43170
previous             43170
poutcome             43170
emp.var.rate         43170
cons.price.idx       43641
cons.conf.idx        43170
euribor3m            52426
nr.employed          43170
y                    43170
date                 43418
latitude             43170
longitude            43170
id_                      0
Income               43000
Kidhome              43000
Teenhome             43000
Dt_Customer          43000
NumWebVisitsMonth    43000
dtype: int64


In [4]:
# Reemplazamos los valores nulos por 0
df.fillna(0, inplace=True)
print("✅ Todos los valores nulos han sido reemplazados por 0.")

✅ Todos los valores nulos han sido reemplazados por 0.


In [5]:
# Comprobamos si existen valores nulos
if df.isnull().values.any():
    print("❌ Todavía hay valores nulos en los datos.")
else:
    print("✅ Todos los valores nulos han sido reemplazados por 0.")

# Comprobamos y verificamos que no existen valores nulos
print(df.isnull().sum())

✅ Todos los valores nulos han sido reemplazados por 0.
Unnamed: 0           0
age                  0
job                  0
marital              0
education            0
default              0
housing              0
loan                 0
contact              0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
date                 0
latitude             0
longitude            0
id_                  0
Income               0
Kidhome              0
Teenhome             0
Dt_Customer          0
NumWebVisitsMonth    0
dtype: int64


#### 4. Revisamos y cambiamos el tipo de dato de cada columna que sea necesario, para obtener una buena práctica, de manera que cada columna, tenga el tipo de dato adecuado

In [6]:
# Convertimos 'age' a int
df['age'] = df['age'].astype(int)

# Convertimos 'default', 'housing', y 'loan' a bool
df['default'] = df['default'].astype(bool)
df['housing'] = df['housing'].astype(bool)
df['loan'] = df['loan'].astype(bool)

# Convertimos 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed' a float64. Coerce se encarga de añadir valores NaN
df['cons.price.idx'] = pd.to_numeric(df['cons.price.idx'], errors='coerce')
df['cons.conf.idx'] = pd.to_numeric(df['cons.conf.idx'], errors='coerce')
df['euribor3m'] = pd.to_numeric(df['euribor3m'], errors='coerce')
df['nr.employed'] = pd.to_numeric(df['nr.employed'], errors='coerce')

# Convertimos 'y' a bool o categoría si es binaria
df['y'] = df['y'].map({'yes': True, 'no': False})

# Convertirmos 'date' a datetime64, especificamos el formato de fecha para que no genere conflicto
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')

# Convertimos 'id_' a string, puesto que es más eficiente en Pandas
df['id_'] = df['id_'].astype('string')

# Convertimos 'Kidhome', 'Teenhome' a int, para no permitir valores decimales
df['Kidhome'] = df['Kidhome'].fillna(0).astype('int')
df['Teenhome'] = df['Teenhome'].fillna(0).astype('int')

# Usamos el errors='coerce', para convertir valores no válidos en NaT
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce')

# Convertimos 'NumWebVisitsMonth' a int, para evitar valores decimales
df['NumWebVisitsMonth'] = df['NumWebVisitsMonth'].fillna(0).astype('int')

# Verificamos los tipos de datos después de los cambios
print(df.dtypes)

Unnamed: 0                    int64
age                           int64
job                          object
marital                      object
education                    object
default                        bool
housing                        bool
loan                           bool
contact                      object
duration                    float64
campaign                    float64
pdays                       float64
previous                    float64
poutcome                     object
emp.var.rate                float64
cons.price.idx              float64
cons.conf.idx               float64
euribor3m                   float64
nr.employed                 float64
y                            object
date                 datetime64[ns]
latitude                    float64
longitude                   float64
id_                  string[python]
Income                      float64
Kidhome                       int64
Teenhome                      int64
Dt_Customer          datetim

#### 5. Realizamos la estandarización de valores categóricos, para evitar posibles conflictos que nos pueda causar el set de datos. Esto es útil a la hora de una búsqueda específica de valores, que contengan espacios y causen conflicto

In [7]:
for columna in df.select_dtypes(include=['object']).columns:
    # Rellenar NaN con una cadena vacía y luego aplicar las transformaciones
    df[columna] = df[columna].fillna('').astype(str).str.strip().str.lower()

# Verificamos que se aplicaron los cambios
print(df.head())

   Unnamed: 0  age        job  marital    education  default  housing   loan  \
0           0    0  housemaid  married     basic.4y    False    False  False   
1           1   57   services  married  high.school    False    False  False   
2           2   37   services  married  high.school    False     True  False   
3           3   40     admin.  married     basic.6y    False    False  False   
4           4   56   services  married  high.school    False    False   True   

     contact  duration  ...      y  date  latitude longitude  \
0  telephone     261.0  ...  false   NaT    41.495   -71.233   
1  telephone     149.0  ...  false   NaT    34.601   -83.923   
2  telephone     226.0  ...  false   NaT    34.939   -94.847   
3  telephone     151.0  ...  false   NaT    49.041   -70.308   
4  telephone     307.0  ...  false   NaT    38.033  -104.463   

                                    id_  Income  Kidhome  Teenhome  \
0  089b39d8-e4d0-461b-87d4-814d71e0e079     0.0        0        

#### 6. Modificamos el nombre de las columnas necesarias, de modo que las columnas que contienen un punto (.) en su nombre, generan conflicto cuando hacemos la llamada dentro de una función

In [8]:
# Modificamos las columnas que poseen caracteres que pueden causar conflicto
df.rename(columns={'emp.var.rate': 'emp_var_rate'}, inplace=True)
df.rename(columns={'cons.price.idx': 'cons_price_idx'}, inplace=True)
df.rename(columns={'cons.conf.idx': 'cons_conf_idx'}, inplace=True)
df.rename(columns={'nr.employed': 'nr_employed'}, inplace=True)

# Modificamos el nombre de esta columna, para mantener la misma estructura en todas, es decir, guión bajo entre cada palabra.
df.rename(columns={'NumWebVisitsMonth': 'Num_Web_Visits_Month'}, inplace=True)

#### 7. Verificamos que se han realizado todas las transformaciones

In [9]:
df.dtypes

Unnamed: 0                       int64
age                              int64
job                             object
marital                         object
education                       object
default                           bool
housing                           bool
loan                              bool
contact                         object
duration                       float64
campaign                       float64
pdays                          float64
previous                       float64
poutcome                        object
emp_var_rate                   float64
cons_price_idx                 float64
cons_conf_idx                  float64
euribor3m                      float64
nr_employed                    float64
y                               object
date                    datetime64[ns]
latitude                       float64
longitude                      float64
id_                     string[python]
Income                         float64
Kidhome                  